In [4]:
import boto3
import os
import zipfile
from PIL import Image
from botocore.exceptions import ClientError

bucket_name = 'art.forgery.capstone.project'

s3 = boto3.client('s3')
try:
    print("Checking bucket ...")
    s3.head_bucket(Bucket=bucket_name)
except ClientError as e:
    print("Bucket not existing, creating it ...")
    s3.create_bucket(Bucket=bucket_name, ACL="private", CreateBucketConfiguration={'LocationConstraint': 'EU'})

zip_dataset = "Cracks2.zip"
tmp_dest = os.path.join(os.path.dirname('data_preprocessing.ipynb'), "tmp")
data_dir = os.path.join(os.path.dirname('data_preprocessing.ipynb'), "cracks")

if not os.path.exists(tmp_dest):
    os.mkdir(tmp_dest)

if len(os.listdir(tmp_dest)) <= 0:
    print("Unzipping archive ...")
    with zipfile.ZipFile(zip_dataset, 'r') as zip_ref:
        zip_ref.extractall(tmp_dest)

dutch_dir = tmp_dest + "/Dutch"
flemish_dir = tmp_dest + "/Flemish"

train_dir = data_dir + "/train/"
validation_dir = data_dir + "/validation/"

print("Checking directories ...")
for dir in [data_dir, train_dir, validation_dir]:
    if not os.path.exists(dir):
        os.mkdir(dir)

Checking bucket ...
Unzipping archive ...
Checking directories ...


In [5]:
def move_and_check_files(origin, dest_train, dest_valid, quantity_train=800, quantity_valid=200):
    moved = 0
    for file in os.listdir(origin):
        try:
            # Before moving the file we check if the image is readable
            im = Image.open(os.path.join(origin, file))
            im.verify()
            os.rename(
                os.path.join(origin, file),
                os.path.join(dest_train if moved < quantity_train else dest_valid, file)
            )
            moved += 1
        except Exception as e:
            print("Bad image : %s" % file)

        if moved >= quantity_train + quantity_valid:
            break

print("Organizing and checking files ...")
for category in [(dutch_dir, 'dutch'), (flemish_dir, 'flemish')]:
    train = train_dir + category[1]
    valid = validation_dir + category[1]
    os.mkdir(train)
    os.mkdir(valid)
    move_and_check_files(category[0], train, valid)

print("Done !")

Organizing and checking files ...


FileNotFoundError: [Errno 2] No such file or directory: 'tmp/Dutch'